In [1]:
import pandas as pd
import os
os.chdir('C:/Users/nxf55806/Documents/Personal/Github/ipl2020/')

matches = pd.read_csv("Raw_Data/kaggle2_matches.csv")
balls = pd.read_csv("Raw_Data/kaggle2_deliveries.csv")

In [2]:
df = pd.merge(matches[['id', 'season', 'city', 'venue']],
              balls[['match_id', 'inning', 'batting_team', 'bowling_team', 'over', 'batsman', 
                     'non_striker', 'bowler', 'batsman_runs']],
              left_on='id', right_on='match_id', how='right')

df.drop(columns=['id', 'venue', 'match_id'], inplace=True)

In [5]:
df.dtypes

season           int64
city            object
inning           int64
batting_team    object
bowling_team    object
over             int64
batsman         object
non_striker     object
bowler          object
batsman_runs     int64
dtype: object

In [10]:
df['season'].unique()

array([2017, 2008, 2009, 2010, 2011, 2012, 2013, 2014, 2015, 2016, 2018,
       2019], dtype=int64)

In [7]:
str_cols = list(df.dtypes[(df.dtypes == object)].index)
df1 = pd.concat([df, pd.get_dummies(df[str_cols], drop_first=True)], axis=1)
df1 = df1.drop(columns = str_cols)

In [13]:
df_train = df1[df1['season'] == 2019]
df_test = df1[df1['season'] != 2019]

X_train = df_train.drop(columns = ['batsman_runs', 'season'])
y_train = df_train['batsman_runs']

X_test = df_test.drop(columns = ['batsman_runs', 'season'])
y_test = df_test['batsman_runs']

In [14]:
import statsmodels.api as sm
mod = sm.OLS(y_train, X_train)
fii = mod.fit()
p_values = fii.summary2().tables[1]['P>|t|']
table = fii.summary2().tables[1]
table

C:\Users\nxf55806\AppData\Local\Continuum\anaconda3\lib\site-packages\statsmodels\base\model.py:1294: RuntimeWarning: invalid value encountered in true_divide
  return self.params / self.bse
C:\Users\nxf55806\AppData\Local\Continuum\anaconda3\lib\site-packages\scipy\stats\_distn_infrastructure.py:901: RuntimeWarning: invalid value encountered in greater
  return (a < x) & (x < b)
C:\Users\nxf55806\AppData\Local\Continuum\anaconda3\lib\site-packages\scipy\stats\_distn_infrastructure.py:901: RuntimeWarning: invalid value encountered in less
  return (a < x) & (x < b)
C:\Users\nxf55806\AppData\Local\Continuum\anaconda3\lib\site-packages\scipy\stats\_distn_infrastructure.py:1892: RuntimeWarning: invalid value encountered in less_equal
  cond2 = cond0 & (x <= _a)


In [15]:
from sklearn.linear_model import LinearRegression

lr = LinearRegression()
lr.fit(X_train, y_train)
y_train_pred = lr.predict(X_train)
y_test_pred = lr.predict(X_test)

In [17]:
from sklearn import metrics
metrics.r2_score(y_train, y_train_pred)    
metrics.r2_score(y_test, y_test_pred)    

-6.317589283869168e+19

In [ ]:

from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import GridSearchCV

cv_grid = {'n_estimators': [50, 200, 500],
               'max_features': [20, 30, 40],
               'max_depth': [25, 50],
               'min_samples_split': [2, 4, 8],
               'min_samples_leaf': [3, 5, 8],
               'bootstrap': [True]}

cv_grid_2 = {'n_estimators': [100, 250, 500],
               'max_features': [25, 60, 120, 250],
               'max_depth': [50, 100],
               'min_samples_split': [2, 4],
               'min_samples_leaf': [1, 3, 5],
               'bootstrap': [True]}

rf = RandomForestRegressor()
rf_cv = GridSearchCV(rf, cv_grid, verbose = 10, n_jobs = -1)
rf_cv.fit(X_train, y_train)
rf_cv.best_params_

rf_cv_2 = GridSearchCV(rf, cv_grid_2, verbose = 10, n_jobs = -1)
rf_cv_2.fit(X_train, y_train)
rf_cv_2.best_params_

rf = RandomForestRegressor(n_estimators = 500, max_features= 20, max_depth= 50,
                           min_samples_split= 8, min_samples_leaf= 8, bootstrap= True)
rf.fit(X_train, y_train)
y_train_pred = rf.predict(X_train)
y_test_pred = rf.predict(X_test)

def mean_absolute_percentage_error(y_true, y_pred): 
    return np.mean(np.abs((y_true - y_pred) / y_true)) * 100

metrics.r2_score(y_train, y_train_pred)
metrics.r2_score(y_test, y_test_pred)    
mean_absolute_percentage_error(y_train, y_train_pred)
mean_absolute_percentage_error(y_test, y_test_pred)    

df_test['prediction'] = y_test_pred
df_test.sort_values('prediction', ascending=False, inplace=True)
